# Symmetric Eigenvalue Decomposition - Lanczos Method
---

If the matrix $A$ is large and sparse and/or if only some
eigenvalues and their eigenvectors are desired, iterative methods are
the methods of choice. For example, the power method can be useful to compute the
eigenvalue with the largest modulus. The basic
operation in the power method is matrix-vector multiplication, and this can be
performed very fast if $A$ is sparse. Moreover, $A$ need not be stored in the
computer --- the input for the algorithm can be just a function which,
given some vector $x$, returns the product $Ax$.

An _improved_ version of the power method, which efficiently computes
some eigenvalues (either largest in modulus or near some target value $\mu$)
and the corresponding eigenvectors, is the Lanczos method.

For more details, see 
[I. Slapničar, Symmetric Matrix Eigenvalue Techniques][Hog14]
and the references therein.

[Hog14]: #1 "L. Hogben, ed., 'Handbook of Linear Algebra', pp. 55.1-55.25, CRC Press, Boca Raton, 2014."


## Prerequisites

The reader should be familiar with concepts of eigenvalues and eigenvectors, related perturbation theory, and algorithms. 

 
## Competences 

The reader should be able to recognise matrices which warrant use uf Lanczos method, to apply 
the method and to assess the accuracy of the solution.

## Lanczos method

$A$ is a real symmetric matrix of order $n$.

### Definitions

Given a nonzero vector $x$ and an index $k<n$, the __Krylov matrix__ is defined as
$K_k=\begin{bmatrix} x & Ax & A^2 x &\cdots & A^{k-1}x \end{bmatrix}$.

__Krilov subspace__ is the subspace spanned by the columns of $K_k$.

### Facts

1. The Lanczos method is based on the following observation. If $K_k=XR$ is the
  $QR$ factorization of the matrix $K_k$, 
  then the $k\times k$ matrix $T=X^T A X$ is tridiagonal. The matrices $X$ and
  $T$ can be computed by using only matrix-vector products in $O(kn)$
  operations.

2. Let $T=Q\Lambda Q^T$ be the EVD of $T$. Then $\lambda_i$ approximate well some of the largest 
and smallest eigenvalues of $A$, and the columns of the matrix $U=XQ$ approximate the corresponding
  eigenvectors.

3. As $k$ increases, the largest (smallest) eigenvalues of the matrix
  $T_{1:k,1:k}$ converge towards some of the largest (smallest) eigenvalues of $A$ (due to
  the Cauchy interlace property). The algorithm can be redesigned to compute
  only largest or smallest eigenvalues. Also, by using shift and invert
  strategy, the method can be used to compute eigenvalues near some specified
  value. In order to obtain better approximations, $k$ should be greater than
  the number of required eigenvalues. On the other side, in order to obtain
  better accuracy and efficacy, $k$ should be as small as possible.

4. The last computed element, $\mu=T_{k+1,k}$, provides
  information about accuracy:
  \begin{align*}
  \|AU-U\Lambda\|_2&=\mu, \\
  \|AU_{:,i}-\lambda_i U_{:,i}\|_2&=\mu |Q_{ki}|, \quad  i=1,\ldots,k.
  \end{align*}
  Further, there are $k$
  eigenvalues $\tilde\lambda_1,\ldots,\tilde\lambda_k$ of $A$ such that
  $|\lambda_i-\tilde\lambda_i|\leq \mu$, and for the corresponding eigenvectors, we
  have $$\sin2\Theta(U_{:,i},\tilde U_{:,i}) \leq \frac{2\mu}{\min_{j\neq i} 
|\lambda_i-\tilde \lambda_j|}.$$ 

5. In practical implementations, $\mu$ is usually used to determine the index $k$. 

6. The Lanczos method has inherent
  numerical instability in the floating-point arithmetic: since the Krylov vectors are, in fact,
  generated by the power method, they converge towards an eigenvector of $A$. 
  Thus, as $k$ increases, the Krylov vectors become more and more parallel, and the recursion in the 
  function `myLanczos()` becomes numerically unstable and the computed columns of $X$
  cease to be sufficiently orthogonal. This affects both the convergence and
  the accuracy of the algorithm. For example, several eigenvalues of $T$ may
  converge towards a simple eigenvalue of $A$ (the, so
  called, _ghost eigenvalues_).

7. The loss of orthogonality is dealt with by using the __full
  reorthogonalization__ procedure: in each step, the new ${\bf z}$ is orthogonalized against all
previous
  columns of $X$, that is, in function `myLanczos()`, the formula 
  ```
  z=z-Tr.dv[i]*X[:,i]-Tr.ev[i-1]*X[:,i-1]
  ```
  is replaced 
  by
  ```
  z=z-sum(dot(z,Tr.dv[i])*X[:,i]-Tr.ev[i-1]*X[:,i-1]
  ```
  To obtain better orthogonality, the latter formula is usually executed twice. 
  The full reorthogonalization raises the operation count to $O(k^2n)$. 
8. The __selective reorthogonalization__ is the procedure in which the current $z$
  is orthogonalized against some selected columns of $X$, in order to
  attain sufficient numerical stability and not increase the operation count
  too much. The details are very subtle and can be found in the references.
  
9. The Lanczos method is usually used for sparse matrices. Sparse matrix $A$
  is stored in the sparse format in which only values and indices of nonzero elements
  are stored. The number of operations required to multiply some vector by $A$ is
  also proportional to the number of nonzero elements.
  
10. The function `eigs()` implements Lanczos method real for symmetric matrices and more general Arnoldi method 
for general matrices.

### Examples

In [1]:
function myLanczos{T}(A::Array{T}, x::Vector{T}, k::Int)
    n=size(A,1)
    X=Array(T,n,k)
    dv=Array(T,k)
    ev=Array(T,k-1)
    X[:,1]=x/norm(x)
    for i=1:k-1
        z=A*X[:,i]
        dv[i]=X[:,i]⋅z
        # Three-term recursion
        if i==1
            z=z-dv[i]*X[:,i]
        else
            # z=z-dv[i]*X[:,i]-ev[i-1]*X[:,i-1]
            # Full reorthogonalization - once or even twice
            z=z-sum([(z⋅X[:,j])*X[:,j] for j=1:i])
            # z=z-sum([(z⋅X[:,j])*X[:,j] for j=1:i])
        end
        μ=norm(z)
        if μ==0
            Tr=SymTridiagonal(dv[1:i-1],ev[1:i-2])
            return eigvals(Tr), X[:,1:i-1]*eigvecs(Tr), X[:,1:i-1], μ
        else
            ev[i]=μ
            X[:,i+1]=z/μ
        end
    end
    # Last step
    z=A*X[:,end]
    dv[end]=X[:,end]⋅z
    z=z-dv[end]*X[:,end]-ev[end]*X[:,end-1]
    μ=norm(z)
    Tr=SymTridiagonal(dv,ev)
    eigvals(Tr), X*eigvecs(Tr), X, μ
end

myLanczos (generic function with 1 method)

In [17]:
n=100
A=full(Symmetric(rand(n,n)))
# Or: A = rand(5,5) |> t -> t + t'
x=rand(n)
k=10

10

In [18]:
λ,U,X,μ=myLanczos(A,x,k)

([-5.27687148716034,-4.202738696536976,-3.0282978314806663,-1.4720018273669415,0.18486429476409857,1.773849118515746,3.0573765748946258,4.8160150280688345,5.441607935087995,50.26326563151359],
100x10 Array{Float64,2}:
  0.109673    -0.00976401  -0.188764    …   0.0532321    0.107569 
 -0.190759     0.132978     0.0258555      -0.0409736    0.0974635
  0.158745     0.219262    -0.0720484      -0.0413919    0.0910975
  0.0851368    0.096687     0.00647746      0.17544      0.0855535
 -0.127815     0.058142    -0.0232598       0.0243537    0.0963576
 -0.0574995   -0.121113     0.114132    …   0.176219     0.100051 
 -0.0852974    0.0407405    0.111498        0.0692991    0.0925069
 -0.106301     0.0811942   -0.00259222      0.113696     0.0965329
 -0.0103438   -0.0310161   -0.204634        0.079871     0.101885 
 -0.0245859    0.0777067    0.0341622      -0.0334475    0.100677 
 -0.0337778   -0.0159161   -0.11983     …   0.0299581    0.0890858
 -0.0132435    0.0881479   -0.0489413      -0

In [19]:
# Orthogonality
X'*X

10x10 Array{Float64,2}:
  1.0          -1.79756e-16   2.37209e-16  …  -1.91514e-16  -1.68985e-16
 -1.79756e-16   1.0           1.15409e-15     -8.83868e-16   2.26103e-16
  2.37209e-16   1.15409e-15   1.0              3.55581e-16   1.50247e-15
  1.91795e-16  -4.55502e-17  -1.10678e-15      9.85084e-16  -2.66133e-16
 -2.2418e-16   -1.07823e-15   1.73564e-16     -2.57103e-16  -1.5604e-15 
 -2.31132e-16  -5.51584e-17   1.35114e-15  …  -1.07124e-15   6.07954e-16
  2.03171e-16   1.03498e-15   1.06961e-17      2.03984e-16   1.44984e-15
  1.68414e-16  -1.79329e-16  -1.27755e-15      1.14117e-15   2.53071e-16
 -1.91514e-16  -8.83868e-16   3.55581e-16      1.0          -1.35937e-15
 -1.68985e-16   2.26103e-16   1.50247e-15     -1.35937e-15   1.0        

In [20]:
X'*A*X

10x10 Array{Float64,2}:
 38.476        21.2754        3.13535e-14  …  -2.95242e-14  -5.60185e-15
 21.2754       11.673         3.07684         -1.55963e-14   4.56382e-15
  3.25118e-14   3.07684       0.0565312        4.76203e-17   7.6232e-17 
  7.23023e-15   5.21382e-16   3.03417          4.99588e-16  -5.36066e-16
 -3.15777e-14  -1.72027e-14  -1.53464e-16      2.23353e-16   1.1587e-16 
 -9.92443e-15  -1.6729e-15    4.66453e-17  …   1.64588e-17  -3.00194e-16
  3.09124e-14   1.71409e-14  -1.12424e-16      6.36025e-16   2.81916e-15
  3.41468e-15  -2.03443e-15   2.95017e-16      2.83593       4.75287e-16
 -2.9645e-14   -1.43718e-14   4.90968e-16     -0.041704      2.40379    
 -2.22244e-15   3.94891e-15   1.85248e-16      2.40379       0.251002   

In [21]:
# Residual
norm(A*U-U*diagm(λ)), μ

(2.848157752821752,2.848157752821752)

In [22]:
U'*A*U

10x10 Array{Float64,2}:
 -5.27687       1.32389e-15   3.65044e-15  …  -2.98003e-15  -1.06979e-14
  1.51867e-15  -4.20274       2.56424e-15     -5.39624e-15   1.52014e-14
  3.57038e-15   2.5704e-15   -3.0283          -1.11189e-15   1.31684e-14
  9.29486e-16  -3.10918e-15   1.63657e-15      3.91893e-15   6.3175e-14 
 -1.42286e-15  -3.41491e-15  -6.56192e-15      5.24125e-15  -9.50726e-15
 -5.22903e-16   2.27726e-15  -3.60642e-15  …   2.27827e-16  -4.03935e-14
 -1.90339e-15  -3.52323e-15  -5.93416e-16     -1.27875e-15   9.19267e-16
 -1.40078e-15   1.51669e-15  -1.67975e-15     -1.06888e-17   6.07715e-15
 -2.90252e-15  -5.4812e-15   -1.33034e-15      5.44161       1.96607e-15
 -1.02668e-14   1.45222e-14   1.49564e-14      1.08516e-15  50.2633     

In [23]:
U'*U

10x10 Array{Float64,2}:
  1.0          -1.94319e-16  -3.16298e-16  …   6.42345e-18  -6.33966e-17
 -1.94319e-16   1.0          -4.56763e-16     -1.53249e-16   2.40324e-16
 -3.16298e-16  -4.56763e-16   1.0              1.21406e-16   6.68617e-17
  2.36967e-16   3.39768e-16   2.41875e-16     -2.85496e-17   1.02785e-15
  1.18955e-16   9.63862e-16   8.13168e-16      3.53036e-16   3.75245e-17
  8.7887e-17    2.41102e-16  -2.91639e-17  …   5.60639e-16  -9.34203e-16
 -1.77858e-16  -1.19848e-16  -1.9859e-16       9.19487e-17  -1.18946e-16
  7.93029e-17  -1.92603e-17   4.64391e-17     -1.22176e-15   1.97563e-16
  6.42345e-18  -1.53249e-16   1.21406e-16      1.0           7.50816e-17
 -6.33966e-17   2.40324e-16   6.68617e-17      7.50816e-17   1.0        

In [24]:
λeig,Ueig=eig(A);

In [25]:
?eigs

search: eigs λeigs Ueigs eigvecs eigvals eigvals! leading_ones leading_zeros



```rst
..  eigs(A, B; nev=6, ncv=max(20,2*nev+1), which="LM", tol=0.0, maxiter=300, sigma=nothing, ritzvec=true, v0=zeros((0,))) -> (d,[v,],nconv,niter,nmult,resid)

Computes generalized eigenvalues ``d`` of ``A`` and ``B`` using Lanczos or Arnoldi iterations for
real symmetric or general nonsymmetric matrices respectively.

The following keyword arguments are supported:

* ``nev``: Number of eigenvalues
* ``ncv``: Number of Krylov vectors used in the computation; should satisfy ``nev+1 <= ncv <= n`` for real symmetric problems and ``nev+2 <= ncv <= n`` for other problems, where ``n`` is the size of the input matrices ``A`` and ``B``. The default is ``ncv = max(20,2*nev+1)``.
  Note that these restrictions limit the input matrix ``A`` to be of dimension at least 2.
* ``which``: type of eigenvalues to compute. See the note below.

  ========= ======================================================================================================================
  ``which`` type of eigenvalues
  ========= ======================================================================================================================
  ``:LM``   eigenvalues of largest magnitude (default)
  ``:SM``   eigenvalues of smallest magnitude
  ``:LR``   eigenvalues of largest real part
  ``:SR``   eigenvalues of smallest real part
  ``:LI``   eigenvalues of largest imaginary part (nonsymmetric or complex ``A`` only)
  ``:SI``   eigenvalues of smallest imaginary part (nonsymmetric or complex ``A`` only)
  ``:BE``   compute half of the eigenvalues from each end of the spectrum, biased in favor of the high end. (real symmetric ``A`` only)
  ========= ======================================================================================================================

* ``tol``: tolerance (:math:`tol \le 0.0` defaults to ``DLAMCH('EPS')``)
* ``maxiter``: Maximum number of iterations (default = 300)
* ``sigma``: Specifies the level shift used in inverse iteration. If ``nothing`` (default), defaults to ordinary (forward) iterations. Otherwise, find eigenvalues close to ``sigma`` using shift and invert iterations.
* ``ritzvec``: Returns the Ritz vectors ``v`` (eigenvectors) if ``true``
* ``v0``: starting vector from which to start the iterations

``eigs`` returns the ``nev`` requested eigenvalues in ``d``, the corresponding Ritz vectors ``v`` (only if ``ritzvec=true``), the number of converged eigenvalues ``nconv``, the number of iterations ``niter`` and the number of matrix vector multiplications ``nmult``, as well as the final residual vector ``resid``.

.. note:: The ``sigma`` and ``which`` keywords interact: the description of eigenvalues searched for by ``which`` do _not_ necessarily refer to the eigenvalue problem :math:`Av = Bv\lambda`, but rather the linear operator constructed by the specification of the iteration mode implied by ``sigma``.

   =============== ================================== ==================================
   ``sigma``       iteration mode                     ``which`` refers to the problem
   =============== ================================== ==================================
   ``nothing``     ordinary (forward)                 :math:`Av = Bv\lambda`
   real or complex inverse with level shift ``sigma`` :math:`(A - \sigma B )^{-1}B = v\nu`
   =============== ================================== ==================================
```

```rst
..  eigs(A; nev=6, ncv=max(20,2*nev+1), which="LM", tol=0.0, maxiter=300, sigma=nothing, ritzvec=true, v0=zeros((0,))) -> (d,[v,],nconv,niter,nmult,resid)

Computes eigenvalues ``d`` of ``A`` using Lanczos or Arnoldi iterations for
real symmetric or general nonsymmetric matrices respectively.

The following keyword arguments are supported:

* ``nev``: Number of eigenvalues
* ``ncv``: Number of Krylov vectors used in the computation; should satisfy ``nev+1 <= ncv <= n`` for real symmetric problems and ``nev+2 <= ncv <= n`` for other problems, where ``n`` is the size of the input matrix ``A``. The default is ``ncv = max(20,2*nev+1)``.
  Note that these restrictions limit the input matrix ``A`` to be of dimension at least 2.
* ``which``: type of eigenvalues to compute. See the note below.

  ========= ======================================================================================================================
  ``which`` type of eigenvalues
  ========= ======================================================================================================================
  ``:LM``   eigenvalues of largest magnitude (default)
  ``:SM``   eigenvalues of smallest magnitude
  ``:LR``   eigenvalues of largest real part
  ``:SR``   eigenvalues of smallest real part
  ``:LI``   eigenvalues of largest imaginary part (nonsymmetric or complex ``A`` only)
  ``:SI``   eigenvalues of smallest imaginary part (nonsymmetric or complex ``A`` only)
  ``:BE``   compute half of the eigenvalues from each end of the spectrum, biased in favor of the high end. (real symmetric ``A`` only)
  ========= ======================================================================================================================

* ``tol``: tolerance (:math:`tol \le 0.0` defaults to ``DLAMCH('EPS')``)
* ``maxiter``: Maximum number of iterations (default = 300)
* ``sigma``: Specifies the level shift used in inverse iteration. If ``nothing`` (default), defaults to ordinary (forward) iterations. Otherwise, find eigenvalues close to ``sigma`` using shift and invert iterations.
* ``ritzvec``: Returns the Ritz vectors ``v`` (eigenvectors) if ``true``
* ``v0``: starting vector from which to start the iterations

``eigs`` returns the ``nev`` requested eigenvalues in ``d``, the corresponding Ritz vectors ``v`` (only if ``ritzvec=true``), the number of converged eigenvalues ``nconv``, the number of iterations ``niter`` and the number of matrix vector multiplications ``nmult``, as well as the final residual vector ``resid``.

.. note:: The ``sigma`` and ``which`` keywords interact: the description of eigenvalues searched for by ``which`` do _not_ necessarily refer to the eigenvalues of ``A``, but rather the linear operator constructed by the specification of the iteration mode implied by ``sigma``.

   =============== ================================== ==================================
   ``sigma``       iteration mode                     ``which`` refers to eigenvalues of
   =============== ================================== ==================================
   ``nothing``     ordinary (forward)                 :math:`A`
   real or complex inverse with level shift ``sigma`` :math:`(A - \sigma I )^{-1}`
   =============== ================================== ==================================
```


In [26]:
λeigs,Ueigs=eigs(A; nev=k, which=:LM, ritzvec=true, v0=x)

([50.26326563151358,5.546524897953146,-5.404675652980687,-5.302711652515531,-5.261076706832911,5.256722098135588,5.1145095266141185,-4.958987333207113,-4.813409589530551,4.759048402956428],
100x10 Array{Float64,2}:
 0.107569   -0.0755404    0.00128829  …   0.0301306   -0.00156553
 0.0974635   0.0254684   -0.101532        0.0462507   -0.112582  
 0.0910975   0.0317241    0.0864657      -0.0229516   -0.00924906
 0.0855535  -0.19969      0.127739       -0.280584    -0.01345   
 0.0963576  -0.0212359    0.0384783       0.109743     0.147914  
 0.100051   -0.167278     0.00576532  …  -0.0309961   -0.0671352 
 0.0925069  -0.116203     0.0278243      -0.168988    -0.0943908 
 0.0965329  -0.0105788   -0.11307        -0.160865     0.15391   
 0.101885   -0.0429681   -0.0649871       0.0226848   -0.0767365 
 0.100677    0.0316463    0.0495724       0.15531      0.0745061 
 0.0890858  -0.0296154    0.0829426   …   0.00759329   0.232539  
 0.101274   -0.0268411    0.00601448      0.188283     0.02

In [28]:
[sort(λ) sort(λeigs)]

10x2 Array{Float64,2}:
 -5.27687   -5.40468
 -4.20274   -5.30271
 -3.0283    -5.26108
 -1.472     -4.95899
  0.184864  -4.81341
  1.77385    4.75905
  3.05738    5.11451
  4.81602    5.25672
  5.44161    5.54652
 50.2633    50.2633 

In [31]:
sort(abs(λeig),rev=true)[1:k]

10-element Array{Float64,1}:
 50.2633 
  5.54652
  5.40468
  5.30271
  5.26108
  5.25672
  5.11451
  4.95899
  4.81341
  4.75905

In [32]:
size(A)

(100,100)

We see that `eigs()` computes `k` eigenvalues with largest modulus. What eigenvalues did `myLanczos()` compute?

In [33]:
for i=1:k
    println(minabs(λeig-λ[i]))
end

0.015794780327416547
0.08578989378515178
0.010045576669919232
0.005186104786622403
0.00890869459065155
0.0222310428599366
0.035796937730963485
0.0569666251123957
0.10491696286515761
2.1316282072803006e-14


Conslusion is that the simple implementation of Lanczos is not enough. However, it is fine, when all eigenvalues are computed:

In [34]:
λall,Uall,Xall,μall=myLanczos(A,x,100)

([-5.40468,-5.30271,-5.26108,-4.95899,-4.81341,-4.65204,-4.37177,-4.29214,-4.11695,-4.01687  …  4.32608,4.35515,4.40328,4.53386,4.67284,4.75905,5.11451,5.25672,5.54652,50.2633],
100x100 Array{Float64,2}:
  0.00128829   0.043048    …   0.159571     0.0755404   0.107569 
 -0.101532    -0.146591       -0.146032    -0.0254684   0.0974635
  0.0864657    0.144099       -0.0544155   -0.0317241   0.0910975
  0.127739     0.0539933       0.0100652    0.19969     0.0855535
  0.0384783   -0.0703815       0.165624     0.0212359   0.0963576
  0.00576532  -0.257071    …   0.0958219    0.167278    0.100051 
  0.0278243   -0.0344539       0.0474659    0.116203    0.0925069
 -0.11307     -0.0021461      -0.150233     0.0105788   0.0965329
 -0.0649871    0.0456651      -0.0801366    0.0429681   0.101885 
  0.0495724   -0.0375195       0.0791787   -0.0316463   0.100677 
  0.0829426    0.0140885   …  -0.0449661    0.0296154   0.0890858
  0.00601448  -0.0785093       0.0883603    0.0268411   0.101274 
  0.

In [35]:
norm(A*Uall-Uall*diagm(λall)), norm(λeig-λall)

(1.4110582875069013e-12,5.0958796090757236e-14)

### Operator version

One can use Lanczos method with operator which, given vector `x`, returns the product `A*x`. The principle is illustrated using the approach from the file `test/runtests.jl` 
from the package [IterativeSolvers.jl](https://github.com/JuliaLang/IterativeSolvers.jl).

Alternative is to use function `LinearMap()` from the package
[LinearMaps.jl](https://github.com/Jutho/LinearMaps.jl)

In [36]:
type MyOp{T}
    buf::Matrix{T}
end
import Base: *, size, eltype, issym
*(A::MyOp, x::AbstractVector) = A.buf*x
size(A::MyOp, i) = size(A.buf, i)
size(A::MyOp) = size(A.buf)
eltype(A::MyOp) = eltype(A.buf)
issym(A::MyOp) = issym(A.buf)

issym (generic function with 12 methods)

In [38]:
B=MyOp(A)

MyOp{Float64}(100x100 Array{Float64,2}:
 0.117826  0.298108   0.190089   …  0.632666   0.844583   0.786966 
 0.298108  0.0709934  0.785583      0.603305   0.436918   0.0799318
 0.190089  0.785583   0.285348      0.212011   0.0878781  0.783456 
 0.732417  0.420689   0.114603      0.696772   0.362221   0.286884 
 0.980434  0.191293   0.976732      0.500164   0.180884   0.356221 
 0.717109  0.670671   0.570056   …  0.673968   0.0285811  0.0313252
 0.946011  0.313959   0.397534      0.332368   0.332275   0.563921 
 0.380396  0.0343988  0.704754      0.340156   0.0526354  0.724786 
 0.622677  0.665812   0.585718      0.928794   0.416132   0.201317 
 0.248275  0.347218   0.0199949     0.913432   0.749937   0.109807 
 0.205039  0.0946733  0.838282   …  0.879452   0.704311   0.810275 
 0.90946   0.318293   0.11345       0.0910911  0.955455   0.659383 
 0.685307  0.99798    0.32303       0.702376   0.757336   0.958754 
 ⋮                               ⋱                                 
 0.98266

In [39]:
λB,UB=eigs(B; nev=k, which=:LM, ritzvec=true, v0=x)

([50.26326563151358,5.546524897953146,-5.404675652980687,-5.302711652515531,-5.261076706832911,5.256722098135588,5.1145095266141185,-4.958987333207113,-4.813409589530551,4.759048402956428],
100x10 Array{Float64,2}:
 0.107569   -0.0755404    0.00128829  …   0.0301306   -0.00156553
 0.0974635   0.0254684   -0.101532        0.0462507   -0.112582  
 0.0910975   0.0317241    0.0864657      -0.0229516   -0.00924906
 0.0855535  -0.19969      0.127739       -0.280584    -0.01345   
 0.0963576  -0.0212359    0.0384783       0.109743     0.147914  
 0.100051   -0.167278     0.00576532  …  -0.0309961   -0.0671352 
 0.0925069  -0.116203     0.0278243      -0.168988    -0.0943908 
 0.0965329  -0.0105788   -0.11307        -0.160865     0.15391   
 0.101885   -0.0429681   -0.0649871       0.0226848   -0.0767365 
 0.100677    0.0316463    0.0495724       0.15531      0.0745061 
 0.0890858  -0.0296154    0.0829426   …   0.00759329   0.232539  
 0.101274   -0.0268411    0.00601448      0.188283     0.02

In [40]:
λeigs-λB

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [41]:
# With LinearMaps
using LinearMaps

In [23]:
methods(LinearMap)

# 5 methods for generic function "LinearMap":
LinearMap{T}(A::Union{AbstractArray{T,2},LinearMaps.AbstractLinearMap{T}}) at /home/slap/.julia/v0.4/LinearMaps/src/LinearMaps.jl:63
LinearMap(f::Function, M::Int64) at /home/slap/.julia/v0.4/LinearMaps/src/LinearMaps.jl:65
LinearMap(f::Function, M::Int64, N::Int64) at /home/slap/.julia/v0.4/LinearMaps/src/LinearMaps.jl:65
LinearMap(f::Function, eltype::Type{T}, M::Int64) at /home/slap/.julia/v0.4/LinearMaps/src/LinearMaps.jl:67
LinearMap(f::Function, eltype::Type{T}, M::Int64, N::Int64) at /home/slap/.julia/v0.4/LinearMaps/src/LinearMaps.jl:67

In [43]:
# Operator from the matrix
C=LinearMap(A)

LinearMaps.WrappedMap{Float64}(100x100 Array{Float64,2}:
 0.117826  0.298108   0.190089   …  0.632666   0.844583   0.786966 
 0.298108  0.0709934  0.785583      0.603305   0.436918   0.0799318
 0.190089  0.785583   0.285348      0.212011   0.0878781  0.783456 
 0.732417  0.420689   0.114603      0.696772   0.362221   0.286884 
 0.980434  0.191293   0.976732      0.500164   0.180884   0.356221 
 0.717109  0.670671   0.570056   …  0.673968   0.0285811  0.0313252
 0.946011  0.313959   0.397534      0.332368   0.332275   0.563921 
 0.380396  0.0343988  0.704754      0.340156   0.0526354  0.724786 
 0.622677  0.665812   0.585718      0.928794   0.416132   0.201317 
 0.248275  0.347218   0.0199949     0.913432   0.749937   0.109807 
 0.205039  0.0946733  0.838282   …  0.879452   0.704311   0.810275 
 0.90946   0.318293   0.11345       0.0910911  0.955455   0.659383 
 0.685307  0.99798    0.32303       0.702376   0.757336   0.958754 
 ⋮                               ⋱                         

In [44]:
λC,UC=eigs(C; nev=k, which=:LM, ritzvec=true, v0=x)
λeigs-λC

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

Here is an example of `LinearMap()` with the function. 

In [45]:
f(x)=A*x

f (generic function with 1 method)

In [46]:
D=LinearMap(f,n,issym=true)

FunctionMap{Float64}(f,100,100;ismutating=false,issym=true,ishermitian=true,isposdef=false,transpose=nothing,ctranspose=nothing)

In [47]:
λD,UD=eigs(D, nev=k, which=:LM, ritzvec=true, v0=x)
λeigs-λD

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

### Sparse matrices

In [48]:
?sprand

search: sprand sprandn sprandbool StepRange



```rst
..  sprand([rng,] m,n,p [,rfn])

Create a random ``m`` by ``n`` sparse matrix, in which the probability of any
element being nonzero is independently given by ``p`` (and hence the mean
density of nonzeros is also exactly ``p``). Nonzero values are sampled from
the distribution specified by ``rfn``. The uniform distribution is used in
case ``rfn`` is not specified. The optional ``rng`` argument specifies a
random number generator, see :ref:`Random Numbers <random-numbers>`.
```


In [49]:
C=sprand(n,n,0.05) |> t -> t+t'

100x100 sparse matrix with 987 Float64 entries:
	[6  ,   1]  =  0.922335
	[11 ,   1]  =  0.522906
	[18 ,   1]  =  0.159718
	[19 ,   1]  =  0.216878
	[20 ,   1]  =  0.348149
	[39 ,   1]  =  0.340006
	[48 ,   1]  =  0.167521
	[60 ,   1]  =  0.218595
	[66 ,   1]  =  0.711681
	[72 ,   1]  =  0.189512
	⋮
	[16 , 100]  =  0.269451
	[17 , 100]  =  0.925692
	[43 , 100]  =  0.36172
	[44 , 100]  =  0.617311
	[51 , 100]  =  0.433796
	[55 , 100]  =  0.215186
	[56 , 100]  =  0.00427824
	[75 , 100]  =  0.577307
	[84 , 100]  =  0.979986
	[91 , 100]  =  0.683289
	[97 , 100]  =  0.101906

In [50]:
issym(C)

true

In [52]:
eigs(C; nev=k, which=:LM, ritzvec=true, v0=x)

([5.8271569304804816,3.7783397581287197,3.6507259244847203,-3.527944171049588,-3.444240194965295,3.29537619272504,-3.2451442530690375,-3.2086453932943573,3.124722281365609,-3.0996019624362194],
100x10 Array{Float64,2}:
 -0.0820151   0.0416868   -0.0941724   …   0.0228322   -0.0279594 
 -0.0689277  -0.0440945    0.044463       -0.0314198   -0.0255235 
 -0.110346    0.0280413   -0.0609073      -0.100101    -0.0986139 
 -0.105342   -0.0387195    0.0227134       0.146785     0.0988668 
 -0.133104   -0.2034       0.0376009       0.202036    -0.0330906 
 -0.0687635   0.0327305   -0.00772304  …   0.0423104    0.100616  
 -0.0822064   0.0377435   -0.114877       -0.0745396    0.0507409 
 -0.0617749  -0.0606303    0.0571659      -0.0182076    0.0742771 
 -0.0464705   0.0020825   -0.0223732      -0.0417644   -0.0545903 
 -0.0674556  -0.0271213    0.0582503       0.0454612   -0.117838  
 -0.0306844   0.0262834   -0.0559147   …   0.00344371   0.00944134
 -0.0587686   0.0409653    0.0871631      -0